In [1]:
import datetime
print('Last Updated On: ', datetime.datetime.now())
print('Author: Philipe Bujold')

Last Updated On:  2022-12-01 07:51:42.513167
Author: Philipe Bujold


In [2]:
import numpy as np
import pandas as pd
# pd.set_option("display.max_columns", None)
# pd.set_option("display.max_rows", None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)
import seaborn as sb 
cm = sb.diverging_palette(5, 250, as_cmap=True)

# import surveyAnalysis as survey #custom analysis scripts
import pyreadstat
import statsmodels.api as sm
from statsmodels.stats.weightstats import DescrStatsW

import plotly.express as px
import plotly.io as pio
pio.renderers.default = "notebook"
from plotly.subplots import make_subplots


from scipy.stats import pearsonr

import surveyAnalysis as survey

In [3]:
from itables import init_notebook_mode, show
init_notebook_mode(all_interactive=False, connected=True)
import itables.options as opt
opt.column_filters = "footer"
opt.lengthMenu = [5, 10, 20, 50, 100, 200, 500]

c:\Anaconda\envs\pandas-profiling2\lib\site-packages\itables\javascript.py:63: UserWarning:

Did you know? init_notebook_mode(all_interactive=False, connected=True) does nothing. Feel free to remove this line, or pass warn_if_call_is_superfluous=False.



In [4]:
spss1, meta = pyreadstat.read_sav(r"C:\Users\pbujold\Rare\Save the Bay - Documents\Data Analysis\spss.sav")#, apply_value_formats=True)
spss1_enriched = pyreadstat.set_value_labels(spss1, meta, formats_as_category=True, formats_as_ordered_category=True)
spss1_enriched['Recruitment'] = "Lucid"
meta.column_names_to_labels['Recruitment'] = 'Platform used to recruit participant'

spss2, meta2 = pyreadstat.read_sav(r"C:\Users\pbujold\Rare\Save the Bay - Documents\Data Analysis\spss2.sav")#, apply_value_formats=True)
spss2_enriched = pyreadstat.set_value_labels(spss2, meta2, formats_as_category=True, formats_as_ordered_category=True)
spss2_enriched['Recruitment'] = "QR Code"

df = pd.concat((spss1_enriched, spss2_enriched))

In [5]:
# Change to numeric variables
for col in ['var24', 'var69', 'var65', 'var212', 'var246', 'var127', 'var68', 'var62']:
    # print(df[col].unique())
    df[col] = df[col].apply(pd.to_numeric, errors='coerce')
        

# Change to unordered 
for col in ['var184', 'var187', 'var175', 'var188', 'var179', 'var185', 'var178', 'var177']:
    df[col] = df[col].astype("category").cat.as_unordered()

# Change to unordered 
for col in ['var9', 'var11', 'var16', 'var241', 'var152', 'var44', 'var85', 'Recruitment' ]:
    df[col] = df[col].astype("category").cat.as_unordered()

#filter for country (i.e. removing Phil's tests)

# df['VGeoCity'].value_counts()
# df['VGeoRegion'].value_counts()
# df['Vpostal'].value_counts()
df = df.loc[df['VGeoCountry'] == 'United States']
df = df.loc[df['var241'].notna()] #some answers don't have the town, probably early practice 
df = df.loc[~df['Vsessionid'].str.contains('test')] #remove any "test" entries

def validateZip(data):
    if data['var16'] == "02842":
        return data['var241'] == "Middletown"
    elif data['var16'] == "02840" or data['var16'] == "02841":
        return data['var241'] == "Newport"
    elif data['var16'] == "02871":
        return data['var241'] == "Portsmouth"
    else:
        return False

df['Valid ZipTown'] = df[['var16', 'var241']].apply(validateZip, axis=1)
meta.column_names_to_labels['Valid ZipTown'] = "Does the respondent's Zip Code match with the town they say they live?"

for col in ['var108', 'var109', 'var110', 'var111', 'var112', 'var113', 'var114', 'var115', 'var116', 'var117', 'var118', 'var119']: 
    df[col] = df[col].replace('I dont know', np.nan)

## 2. Respondent's Lawn Care Practices
___

In [14]:
import plotly.graph_objects as go
fig = go.Figure(data=[go.Pie(labels=df[['var37O88']].value_counts().index, values=df[['var37O88']].value_counts().values, textinfo='label+percent')])
fig.update_layout(
width=1000,
height=450,
title={
    "text": meta.column_names_to_labels['var37O88'],
    "x": 0.5,
    "xanchor": "center",
    "font": dict(
                family="Arial",
                size=12,
            ),
    },
# xaxis_tickangle=45,  # , "x": 0.5, "xanchor": "center"},
)

fig.show()

In [15]:

carePlot = survey.plot_single_proportions_2( df[[ 'var44', 'var49', 'var48', 'var52']],
                                                 meta,
                                                 orientation='v')

arePlot = survey.plot_single_proportions_2( df[['var37O86',
                                                'var37O87',
                                                'var37O88',
                                                'var37O89',
                                                'var37O90',
                                                'var37O91',
                                                'var37O92']],
                                                 meta,
                                                 orientation='h')

arePlot = survey.plot_single_proportions_2( df[[
                                                'var58O195',
                                                'var58O196',
                                                'var58O197',
                                                'var58O198',
                                                'var58O199',
                                                'var58O200',
                                                'var58O201',
                                                'var58O202',
                                                'var58O203',
                                                'var58O204',
                                                'var58O205',
                                                'var58O206',
                                                'var58O207',
                                                'var58O209',
                                                'var58O210',
                                                "var58O208",
                                                ]],
                                                 meta,
                                                 orientation='h')

arePlot = survey.plot_single_proportions_2( df[[
'var50O158',
 'var50O159',
 'var50O160',
 'var50O161',
 'var50O162',
 'var50O163',
 'var50O164',
 'var50O165',
                                                ]],
                                                 meta,
                                                 orientation='h')


arePlot = survey.plot_single_proportions_2( df[[
'var56O185',
 'var56O186',
 'var56O187',
 'var56O188',
 'var56O189',
 'var56O914',
 'var56O190',
                                                ]],
                                                 meta,
                                                 orientation='h')

arePlot = survey.plot_single_proportions_2( df[[
                                                'var51O168',
                                                'var51O169',
                                                'var51O170',
                                                'var51O171',
                                                'var51O172',
                                                'var51O173',
                                                ]],
                                                 meta,
                                                 orientation='h')           

arePlot = survey.plot_single_proportions_2( df[[
'var210O901',
 'var210O902',
 'var210O903',
 'var210O904',
 'var210O905',
 'var210O906',                                      
                                                ]],
                                                 meta,
                                                 orientation='h')      
  
